# Setup (same everywhere)

## Mount Drive


In [1]:
from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%cd drive/'My Drive'/repositories/moleculenet/notebooks

/content/drive/My Drive/repositories/moleculenet/notebooks


In [3]:
%ls

colab_extended_grid_search_pipeline.ipynb
colab_grid_search_pipeline.ipynb
colab_reproducing_grid_search_pipeline.ipynb
colab_reproducing_rf_ci.ipynb
colab_RF_CIs_on_reproducing.ipynb
necessary_eda.ipynb
visualise_extended_grid_search_results.ipynb
visualise_grid_search_results.ipynb
visualise_reproducing_grid_search_results.ipynb


In [4]:
%ls ../data/

esol_original_1024ecfp4_features.csv
esol_original_1024ecfp6_features.csv
esol_original_2048ecfp4_features.csv
esol_original_2048ecfp6_features.csv
esol_original.csv
esol_original_extra_features.csv
esol_original_IdSmilesLabels.csv
esol_original_rdkit_features.csv
ESOL_README
freesolv_original_1024ecfp4_features.csv
freesolv_original_1024ecfp6_features.csv
freesolv_original_2048ecfp4_features.csv
freesolv_original_2048ecfp6_features.csv
freesolv_original.csv
freesolv_original_IdSmilesLabels.csv
freesolv_original_rdkit_features.csv
FreeSolv_README
lipophilicity_original_1024ecfp4_features.csv
lipophilicity_original_1024ecfp6_features.csv
lipophilicity_original_2048ecfp4_features.csv
lipophilicity_original_2048ecfp6_features.csv
lipophilicity_original.csv
lipophilicity_original_IdSmilesLabels.csv
lipophilicity_original_rdkit_features.csv
Lipo_README


## Import modules

### Standard imports

In [5]:
import warnings
warnings.filterwarnings('ignore')

# custom imports
import os
import sys

# saving models
import json
import pickle

# standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# metrics
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import jaccard_score # Tanimoto

# making custom metrics
from sklearn.metrics import make_scorer

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

# preprocessing
from sklearn.feature_selection import VarianceThreshold # to remove zero-var features
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

# models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.neural_network import MLPRegressor

from sklearn.kernel_ridge import KernelRidge

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel
from sklearn.gaussian_process.kernels import RBF, Matern, DotProduct

# pipelines
# https://scikit-learn.org/stable/modules/compose.html#combining-estimators
from sklearn.pipeline import make_pipeline, Pipeline

In [6]:
import sklearn
print(sklearn.__version__)

0.22.2.post1


### Custom imports

In [7]:
sys.path.insert(0, '..')

# global vars
from util_scripts.preprocessing_functions import list_highly_correlated

sys.path.insert(0, './notebooks')

In [8]:
help(list_highly_correlated)

Help on function list_highly_correlated in module util_scripts.preprocessing_functions:

list_highly_correlated(df_features, targets, threshold=0.8)
    List column names of the dataframe of features which are highly correlated
    to the target (absolute value of the correlation is greater than threshold).
    
    df_features: pandas.core.frame.DataFrame of features
    targets: pandas.core.series.Series of targets
    threshold: float in [0, 1] above which we consider a feature highly correlated
    
    returns: list of columns names from df_features, which are highly correlated
             to the target



## Set plotting style

In [9]:
%matplotlib inline
plt.style.use('fivethirtyeight')

plt.rcParams['axes.facecolor']='w'
#plt.rcParams['axes.linewidth']=1
plt.rcParams['axes.edgecolor']='w'
plt.rcParams['figure.facecolor']='w'
plt.rcParams['savefig.facecolor']='w'
#plt.rcParams['grid.color']='white'

# Load Data (same everywhere)

## CHOOSE: dataset, smile_type

In [10]:
dataset = 'lipophilicity'
assert dataset in ['freesolv', 'esol', 'lipophilicity']

smile_type = 'original'
assert smile_type in ['original', 'protonated']

grid_search_type = 'extended'
assert grid_search_type in ['reproducing', 'extended']

## Load Features and Targets

Leave all features here so setup and Load and prepare data are the same everywhere.

In [11]:
# original data
id_smile_target = pd.read_csv(f'../data/{dataset}_{smile_type}_IdSmilesLabels.csv', index_col=0)
# labels
labels = id_smile_target['labels']

In [12]:
# fingerprints
ecfp4_1024_features = pd.read_csv(f'../data/{dataset}_{smile_type}_1024ecfp4_features.csv', index_col=0)
ecfp6_1024_features = pd.read_csv(f'../data/{dataset}_{smile_type}_1024ecfp6_features.csv', index_col=0)

ecfp4_2048_features = pd.read_csv(f'../data/{dataset}_{smile_type}_2048ecfp4_features.csv', index_col=0)
ecfp6_2048_features = pd.read_csv(f'../data/{dataset}_{smile_type}_2048ecfp6_features.csv', index_col=0)

In [13]:
# RDKit descriptors
rdkit_features = pd.read_csv(f'../data/{dataset}_{smile_type}_rdkit_features.csv', index_col=0)

highly_correlated_features = list_highly_correlated(rdkit_features, labels, threshold=0.75)

print(f'\nRemoving {len(highly_correlated_features)} highly correlated feature(s).')
rdkit_features = rdkit_features.drop(highly_correlated_features, axis=1)


Found 0 highly-correlated feature(s):
[]

Removing 0 highly correlated feature(s).


In [14]:
print('rdkit_features.shape:      ', rdkit_features.shape)
print('ecfp4_1024_features.shape: ', ecfp4_1024_features.shape)
print('ecfp6_1024_features.shape: ', ecfp6_1024_features.shape)
print('ecfp4_2048_features.shape: ', ecfp4_2048_features.shape)
print('ecfp6_2048_features.shape: ', ecfp6_2048_features.shape)
print('labels.shape:              ', labels.shape)

rdkit_features.shape:       (4200, 200)
ecfp4_1024_features.shape:  (4200, 1024)
ecfp6_1024_features.shape:  (4200, 1024)
ecfp4_2048_features.shape:  (4200, 2048)
ecfp6_2048_features.shape:  (4200, 2048)
labels.shape:               (4200,)


In [15]:
labels.head()

id
CHEMBL596271     3.54
CHEMBL1951080   -1.18
CHEMBL1771       3.69
CHEMBL234951     3.37
CHEMBL565079     3.10
Name: labels, dtype: float64

## Create one DataFrame with all features

In [16]:
all_features = pd.concat([rdkit_features,
                          ecfp4_1024_features, ecfp6_1024_features,
                          ecfp4_2048_features, ecfp6_2048_features],
                         axis='columns')

In [17]:
all_features.shape

(4200, 6344)

In [18]:
all_features.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,...,2048ecfp6-2008,2048ecfp6-2009,2048ecfp6-2010,2048ecfp6-2011,2048ecfp6-2012,2048ecfp6-2013,2048ecfp6-2014,2048ecfp6-2015,2048ecfp6-2016,2048ecfp6-2017,2048ecfp6-2018,2048ecfp6-2019,2048ecfp6-2020,2048ecfp6-2021,2048ecfp6-2022,2048ecfp6-2023,2048ecfp6-2024,2048ecfp6-2025,2048ecfp6-2026,2048ecfp6-2027,2048ecfp6-2028,2048ecfp6-2029,2048ecfp6-2030,2048ecfp6-2031,2048ecfp6-2032,2048ecfp6-2033,2048ecfp6-2034,2048ecfp6-2035,2048ecfp6-2036,2048ecfp6-2037,2048ecfp6-2038,2048ecfp6-2039,2048ecfp6-2040,2048ecfp6-2041,2048ecfp6-2042,2048ecfp6-2043,2048ecfp6-2044,2048ecfp6-2045,2048ecfp6-2046,2048ecfp6-2047
CHEMBL596271,8.838871,-4.082382,8.838871,0.008322,0.728444,340.858,319.690,340.145474,124.0,0.0,0.123343,-0.368964,0.368964,0.123343,1.000000,1.750000,2.500000,2.123459,1687.752538,34.944711,32.666819,12.422748,20.033375,16.898657,6.776621,4.360460,4.738425,2.865640,3.054623,1.804869,1.899360,-2.04,1.380884e+09,4.567251,6.457627,2.993329,176.320077,9.467009,5.824404,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CHEMBL1951080,15.117958,-6.843264,15.117958,0.920611,0.545587,494.591,468.383,494.118143,178.0,0.0,0.312967,-0.495171,0.495171,0.312967,1.151515,1.939394,2.696970,2.769399,2190.360203,46.566031,41.568662,17.201655,26.024268,20.807038,10.208136,5.392361,7.728262,3.448578,5.643219,2.246019,3.911837,-2.92,2.508526e+11,7.540731,10.344648,5.359304,233.343997,19.892347,16.394507,1.411842,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
CHEMBL1771,13.321227,-3.906276,13.321227,0.250582,0.807761,321.829,305.701,321.059027,110.0,0.0,0.327301,-0.467586,0.467586,0.327301,1.428571,2.285714,3.142857,2.571593,1277.917336,28.842417,26.049923,11.622348,16.467826,13.380423,6.574884,3.644374,4.838835,2.449223,3.447131,1.574234,2.308795,-1.36,3.162085e+07,4.816528,6.178251,2.838691,155.130377,4.736863,6.017892,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CHEMBL234951,14.213604,-4.272077,14.213604,0.187400,0.506650,419.890,401.746,419.070655,146.0,0.0,0.267913,-0.393614,0.393614,0.267913,1.392857,2.250000,3.035714,2.101452,1817.843838,35.436275,31.260847,14.833272,20.860037,15.939898,8.423436,5.064052,6.665194,3.342526,4.438508,2.230475,3.208105,-2.39,3.516594e+09,7.139612,7.555801,3.513611,194.671846,25.404343,16.542217,2.823684,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
CHEMBL565079,14.167882,-4.810274,14.167882,0.671279,0.747686,381.480,354.264,381.216475,148.0,0.0,0.269711,-0.341203,0.341203,0.269711,1.142857,1.928571,2.607143,3.344425,1874.705050,43.997117,40.552565,13.552565,24.094035,20.291530,6.897103,4.765143,4.765143,2.698162,2.698162,1.590024,1.590024,-3.14,3.593066e+10,5.065096,8.698847,5.428227,203.340701,10.619627,18.208680,2.823684,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# CHOOSE: what feature_sets we will iterate over

## Put original features into feature_sets
Create a dictionary containing feature names (index objects) as its elements

In [19]:
feature_sets = {
    'RDKit': rdkit_features.columns,
    '1024ecfp-4': ecfp4_1024_features.columns,
    '1024ecfp-6': ecfp6_1024_features.columns,
    '2048ecfp-4': ecfp4_2048_features.columns,
    '2048ecfp-6': ecfp6_2048_features.columns,
}

## Adding a combination of RDKit features with every feature set in the feature_sets dictionary


In [20]:
for f in ['1024ecfp-4', '1024ecfp-6', '2048ecfp-4', '2048ecfp-6']:
    feature_sets[f'{f} + RDKit'] = feature_sets[f].union(feature_sets['RDKit'])

In [21]:
list(feature_sets.keys())

['RDKit',
 '1024ecfp-4',
 '1024ecfp-6',
 '2048ecfp-4',
 '2048ecfp-6',
 '1024ecfp-4 + RDKit',
 '1024ecfp-6 + RDKit',
 '2048ecfp-4 + RDKit',
 '2048ecfp-6 + RDKit']

In [22]:
[len(feature_sets[key]) for key in feature_sets.keys()]

[200, 1024, 1024, 2048, 2048, 1224, 1224, 2248, 2248]

# Train-Validation-Test split

## CHOOSE: use_small - use 100 observations

In [23]:
# if True, use only 100 observations with 90-10 train-test-split for computational efficiency
use_small = False

In [24]:
if use_small:
    working_size = 100
else:
    working_size = all_features.shape[0]

## CHOOSE: way to do train-val-test splits

In [25]:
def make_split_generator(X, y, split_type='random', random_state=42, n_splits=1, test_size=0.1):
    assert split_type in ['random', 'stratified']

    if split_type == 'random':
        gen = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state).split(X)
    elif split_type == 'stratified':
        binned = y.apply(lambda x: int(x)) # creating stratified indices
        gen = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state).split(X, binned)
    
    # gen for generator
    return gen

### TrainVal-Test split: 90/10

In [26]:
# needed fot creating a dataframe of train_val features (reproduced for testing)
trainval_test_split_gen = make_split_generator(X=all_features.iloc[:working_size],
                                               y=labels[:working_size],
                                               split_type='random', random_state=42,
                                               n_splits=1, test_size=0.1)
#get numeric indexes
train_val, test = next(trainval_test_split_gen)
# get real indexes (i.e. Chembl id, substance name)
train_val, test = all_features.iloc[train_val].index, all_features.iloc[test].index
# we will use them later

print('TrainVal:')
print(train_val[:5], len(train_val))

print('\nTest:')
print(test[:5], len(test))

TrainVal:
Index(['CHEMBL2325714', 'CHEMBL256985', 'CHEMBL298384', 'CHEMBL205807',
       'CHEMBL1652621'],
      dtype='object') 3780

Test:
Index(['CHEMBL1431112', 'CHEMBL1322675', 'CHEMBL2030964', 'CHEMBL1381989',
       'CHEMBL74582'],
      dtype='object') 420


## CHOOSE: feature set to use for now, will itarate later

**This is only used for debugging**

In [27]:
# main feature set to use for now
f = 'RDKit'
assert f in feature_sets

In [28]:
# train_val data frame
features = all_features.loc[train_val, feature_sets[f]]
#train_val targets
targets = labels.loc[train_val]

print(features.shape, targets.shape)

(3780, 200) (3780,)


### Train-Val split: 80/10 (resulting in 80-10-10 in train-val-test)

**This is only used for debugging**

In [29]:
# needed fot creating a dataframe of train_val features (reproduced for testing)
train_val_split_gen = make_split_generator(X=features, y=targets,
                                           split_type='random', random_state=42,
                                           n_splits=1, test_size=1/9)
# get numeric indexes
train, val = next(train_val_split_gen)
# get real indexes (i.e. Chembl id, substance name)
train, val = all_features.iloc[train].index, all_features.iloc[val].index

print('Train:')
print(train[:5], len(train))

print('\nVal:')
print(val[:5], len(val))

Train:
Index(['CHEMBL2035039', 'CHEMBL51776', 'CHEMBL429682', 'CHEMBL452',
       'CHEMBL1822878'],
      dtype='object') 3360

Val:
Index(['CHEMBL460', 'CHEMBL1689118', 'CHEMBL20210', 'CHEMBL320882',
       'CHEMBL1223955'],
      dtype='object') 420


# Training

## CHOOSE: metrics to use

See sklearn documentation 3.3.1.4. Using multiple metric evaluation:


In [30]:
def pearson_corr_coef(y_true, y_pred):
    """
    Original scipy.stats.pearsonr returns a tuple (r, p):
        r : float
            Pearson's correlation coefficient.  
        p-value : float
            Two-tailed p-value.
    """
    return pearsonr(y_true, y_pred)[0]

In [31]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [32]:
scoring = {
    'RMSE': make_scorer(rmse, greater_is_better=False),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'R^2': make_scorer(r2_score, greater_is_better=True),
    'pearson_r': make_scorer(pearson_corr_coef, greater_is_better=True)
}

## CHOOSE: estimators to consider

### From the Paper (estimators):

- 3.5.3 **Kernel ridge regression.** Kernel ridge regression (KRR) is a combination of ridge regression and kernel trick. By using a nonlinear kernel function (**radial basis function**), it learns a non-linear function in the original space that maps features to predicted values.

- 3.5.4 **Random forests.** Random forests (RF) are ensemble prediction methods.(72) A random forest consists of many individual decision trees, each of which is trained on a subsampled version of the original dataset. The results for individual trees are averaged to provide output predictions for the full forest. Random forests can be used for both classification and regression tasks. Training a random forest can be computa- tionally intensive, so benchmarks only include random forest results for smaller datasets.

- 3.5.5 **Gradient boosting.** Gradient boosting is another ensemble method consisting of individual decision trees.(73) In contrast to random forests, it builds relatively simple trees which are sequentially incorporated to the ensemble. In each step, a new tree is generated in a greedy manner to minimize loss function. A sequence of such “weak” trees are combined together into an additive model. We utilize the XGBoost implementation of gradient boosting in DeepChem.(79)

### Extra:

- **Gaussian Processes.** 

In [33]:
estimators = {
    'rf':  RandomForestRegressor(),
    'xgb': XGBRegressor(),
    'krr': KernelRidge(kernel='rbf'), # 'rbf' used in the paper (defaults to 'linear')
    'gp': GaussianProcessRegressor(normalize_y=True), # normilize since we have not normalized here
}

## CHOOSE: hyperparameters to tune

### From the supplementary materials (hyperparameters):

1. **Model Training and Hyperparameter Optimization**
All models were trained on Stanford’s GPU clusters via DeepChem. No model was allowed to train for more than 10 hours(time profile in Table S1. Users can reproduce benchmarks locally by following directions from DeepChem.
Hyperparameters were determined using Gaussian Process Optimization via pyGPGO (https://github.com/hawk31/pyGPGO), with max number of iterations set to 20. Optimized hyperparameters for each model are listed, detailed hyperparameters
can be found on Deepchem.

    1.3 Kernel Ridge Regression (KRR)
        - Penalty parameter
    1.4 Random Forest (RF)
        - Number of trees in the forest: 500
    1.5 Gradient Boosting (XGBoost)
        - Maximum tree depth
        - Learning rate
        - Number of boosted tree
        
### Extra:

    Gaussian Processes (GP):
        - Kernel

In [34]:
params = {
    'rf': {
        'rf__n_estimators': np.arange(50, 1050, 50),
        'rf__max_features': np.arange(0.1, 1.0, 0.1)
    },
    'xgb': {
        'xgb__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
        'xgb__max_depth': np.arange(1, 11, 2),
        'xgb__n_estimators': np.arange(50, 550, 50),
        'xgb__subsample': [0.5, 1]
    },
    'krr': {
        'krr__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
    },
    'gp': {
        'gp__kernel': [RBF() + WhiteKernel(),
                       Matern() + WhiteKernel(),
                       DotProduct() + WhiteKernel()]
    }
}

# save the grid used for grid search 
with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_grid_params.pickle', 'wb') as f:
    pickle.dump(params, f, protocol=pickle.HIGHEST_PROTOCOL)

## CHOOSE: number of iterations

**Set n_iter=20 to be consistent with GP hyperparameter search**

## Gridsearch loop

In [ ]:
%%time

best_params = {}
val_scores = {}
train_test_scores = {}

for f in feature_sets:
    print(f'Using {f} features...')

    # create dataframes of only the features and observarions we are looking at now without na-s
    #
    # train_val data frame
    features = all_features.loc[train_val, feature_sets[f]].dropna(axis=0)
    # train_val targets
    targets = labels.loc[features.index]

    best_params[f] = {}
    val_scores[f] = {}
    train_test_scores[f] = {}


    for e in estimators:
        print(f'\tRandom search optimisation for {e} estimator...')

        # leave like that so that parameter keys keep working
        pipe = Pipeline([('zero-var-feature-remover', VarianceThreshold()), 
                        ('scaler', StandardScaler()),
                         (e, estimators[e])])
        
        # make train/val split generator
        train_val_split_gen = make_split_generator(X=features, y=targets,
                                                   split_type='random', random_state=42,
                                                   n_splits=1, test_size=1/9)

        # fit models and optimize paramerers
            # refit=False: .best_estimator_ is not available, .but best_params_ are
            # scoring='neg_mean_squared_error': equivalent to RMSE (might be faster to use builtin version)
        model = RandomizedSearchCV(pipe, param_distributions=params[e],
                                   cv=train_val_split_gen,
                                   scoring='neg_mean_squared_error',
                                   refit=True, # False: can's use .best_estimator_
                                   n_iter=20, n_jobs=-1,
                                   random_state=42).fit(features, targets)

        # record best model parameters
        best_params[f][e] = model.best_params_

        # save best parameters
        with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_params.pickle', 'wb') as fp:
            pickle.dump(best_params, fp, protocol=pickle.HIGHEST_PROTOCOL)

        # make generator for 3 train/val splits
        train_val_split_gen = make_split_generator(X=features, y=targets,
                                                   split_type='random', random_state=42,
                                                   n_splits=3, test_size=1/9)


        # get metrics for the validation set
        val_results = cross_validate(estimator=model.best_estimator_,
                                     X=features, y=targets,
                                     cv=train_val_split_gen,
                                     scoring=scoring, n_jobs=-1)
        

        # record metrics (validation set) when fitting with best parameters
        val_scores[f][e] = val_results

        # save validation scores
        with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_val_scores.pickle', 'wb') as fp:
            pickle.dump(val_scores, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
        # create dataframes of only the features and observarions we are looking at now without na-s
        all_interesting_features = all_features[feature_sets[f]].iloc[:working_size].dropna(axis=0)
        all_interesting_targets = labels.loc[all_interesting_features.index]

        # make generator for 3 trainval/test splits
        trainval_test_split_gen = make_split_generator(X=all_interesting_features,
                                                       y=all_interesting_targets,
                                                       split_type='random', random_state=42,
                                                       n_splits=3, test_size=0.1)
        
        # get metrics for the train and test set
        #   make sure to restrict feature set here (don't want everything)
        test_results = cross_validate(estimator=model.best_estimator_,
                                      X=all_interesting_features,
                                      y=all_interesting_targets,
                                      cv=trainval_test_split_gen,
                                      scoring=scoring, n_jobs=-1,
                                      return_train_score=True)
        train_test_scores[f][e] = test_results

        # save train test scores
        with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_train_test_scores.pickle', 'wb') as fp:
            pickle.dump(train_test_scores, fp, protocol=pickle.HIGHEST_PROTOCOL)

%store best_params
%store val_scores
%store train_test_scores

Using RDKit features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[15:15:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
	Random search optimisation for gp estimator...
Using 1024ecfp-4 features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[16:22:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
	Random search optimisation for gp estimator...
Using 1024ecfp-6 features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[18:31:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
	Ra

In [ ]:
# retrieve 
%store -r best_params
print(best_params)

In [ ]:
# retrieve 
%store -r val_scores
print(val_scores)

In [ ]:
# retrieve 
%store -r train_test_scores
display(pd.DataFrame(pd.DataFrame(train_test_scores).loc['xgb' ,'1024ecfp-4']))

In [ ]:
val_scores['1024ecfp-4']['rf']['test_RMSE'], type(val_scores['1024ecfp-4']['rf']['test_RMSE'])

# CHOOSE: Names for saving results

## Into Pickle files

In [ ]:
with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_val_scores.pickle', 'wb') as fp:
    pickle.dump(val_scores, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_train_test_scores.pickle', 'wb') as fp:
    pickle.dump(train_test_scores, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_best_params.pickle', 'wb') as fp:
    pickle.dump(best_params, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../results/{dataset}_{smile_type}_{grid_search_type}_random_search_grid_params.pickle', 'wb') as f:
    pickle.dump(params, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(f'../results/{dataset}_{smile_type}_random_search_best_cv_scores.pickle', 'rb') as fp:
#     cv_scores = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_best_test_score.pickle', 'rb') as fp:
#     test_score = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_best_params.pickle', 'rb') as fp:
#     best_params = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_grid_params.pickle', 'rb') as fp:
#     params = pickle.load(fp)

In [ ]:
# cv_scores

In [ ]:
# test_score

In [ ]:
# best_params

In [ ]:
# params